In [ ]:

sample_recipe = pd.DataFrame(sample_recipe)
sample_recipe['id'] = sample_recipe.index
sample_recipe
sample_recipe.drop("Unnamed: 0", axis=1, inplace=True)


# Function to clean ingredient strings
def clean_ingredient(ingredient):
    # Remove special characters and replace spaces with underscores, convert to lowercase
    cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', ingredient).lower().replace(' ', '_')
    return cleaned.strip()


# Function to process and clean the NER column
def process_and_clean_ner_column(ner_list):
    try:
        # Evaluate the string to a list if necessary
        if isinstance(ner_list, str):
            ner_list = eval(ner_list)
        # Ensure ner_list is actually a list
        if isinstance(ner_list, list):
            # Clean each ingredient in the list
            cleaned_ingredients = [clean_ingredient(ingredient) for ingredient in ner_list]
            return cleaned_ingredients
        else:
            return ner_list
    except Exception as e:
        print(f"Error processing list: {e}")
        return ner_list  # Return the original list if there's an error


sample_recipe['NER'] = sample_recipe['NER'].apply(process_and_clean_ner_column)
sample_recipe
sample_recipe.NER = sample_recipe.NER.astype(str)
ingr_mapping_df = ingr_df.copy()
ingr_mapping_df['ingredient_name'] = ingr_mapping_df['ingredient_name'].astype(str)
ingr_mapping_df['ingredient_name'] = ingr_mapping_df['ingredient_name'].astype(str)
ingr_mapping_df
ingredient_id_list = ingr_mapping_df['ingredient_id'].unique()
# Create a DataFrame to hold the new columns with initial values set to 0
new_columns_ingr_df = pd.DataFrame(0, index=ingr_mapping_df.index, columns=ingredient_id_list)

# Concatenate the new columns to ingr_small
sample_recipe = pd.concat([sample_recipe, new_columns_ingr_df], axis=1)
sample_recipe
# Iterate through sample_recipe NER column rows
for index, row in sample_recipe.iterrows():
    ingredients_list = row['NER']
    # Convert the string representation of the list to an actual list if needed
    if isinstance(ingredients_list, str):
        ingredients_list = eval(ingredients_list)
    # Iterate through each value in the ingredients list
    for ingredient in ingredients_list:
        # Check if the ingredient matches any ingredient_name in ingr_mapping_df
        match = ingr_mapping_df[ingr_mapping_df['ingredient_name'] == ingredient]
        if not match.empty:
            ingredient_id = match['ingredient_id'].values[0]
            # Update the corresponding cell to 1
            sample_recipe.at[index, str(ingredient_id)] = 1

# Verify the result
print(sample_recipe.head())
# Identify columns that are compound IDs
ingredient_id_cols = [col for col in sample_recipe.columns if col not in ['id', 'NER']]

# Calculate the number of 1s in each row
sample_recipe['count_ones'] = sample_recipe[ingredient_id_cols].sum(axis=1)

# Plot the number of 1s in each row
plt.figure(figsize=(10, 6))
plt.bar(sample_recipe.index, sample_recipe['count_ones'])
plt.xlabel('Row Index')
plt.ylabel('Number of 1s')
plt.title('Number of 1s in Each Row')
plt.show()

sample_recipe.count_ones.mean()
sample_recipe.count_ones.max()
sample_recipe.count_ones.std()
# Initialize a list to store unmatched ingredients
unmatched_ingredients = []

# Iterate through sample_recipe NER column rows
for index, row in sample_recipe.iterrows():
    ingredients_list = row['NER']
    # Convert the string representation of the list to an actual list if needed
    if isinstance(ingredients_list, str):
        ingredients_list = eval(ingredients_list)
    # Iterate through each value in the ingredients list
    for ingredient in ingredients_list:
        # Check if the ingredient matches any ingredient_name in ingr_mapping_df
        match = ingr_mapping_df[ingr_mapping_df['ingredient_name'] == ingredient]
        if not match.empty:
            ingredient_id = match['ingredient_id'].values[0]
            # Update the corresponding cell to 1
            sample_recipe.at[index, str(ingredient_id)] = 1
        else:
            # If no match, add the ingredient to the unmatched_ingredients list
            unmatched_ingredients.append(ingredient)
unmatched_ingredients = pd.Series(unmatched_ingredients)
unmatched_ingredients.unique()
unmatched_ingredients.nunique()